In [2]:
import pymysql
import pandas as pd
from datetime import datetime, timedelta
import traceback2

In [3]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)

def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


In [4]:
def get_unmapped_coins():
    now = datetime.now()
    today = datetime(now.year, now.month, now.day)
    sql = "select * from notebook_missing_symbols;"
    df = pd.read_sql(sql, __connect__())
    df['reported_on'] = today
    return df
    

In [16]:
df = get_unmapped_coins()

In [17]:
df

,missing_project,data_source,reported_on
0,Clipper,tt,2021-08-16
1,dYdX,tt,2021-08-16
2,Polymarket,tt,2021-08-16
3,MetaMask,tt,2021-08-16
4,Saddle Finance,tt,2021-08-16
...,...,...,...
158,XSwap,Debank,2021-08-16
159,KSwap,Debank,2021-08-16
160,ACMD,Debank,2021-08-16
161,SushiSwapLend,Debank,2021-08-16


In [12]:
def insert_new_coins(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    df = df.astype(object).where(pd.notnull(df), None)
    for i, row in df.iterrows():
        dt = row["reported_on"].timestamp()
        print(row["reported_on"], row["data_source"], row["missing_project"])
        try:
            sql = """INSERT INTO `new_symbols_reporting` (`reported_on`, `data_source`, `missing_project`) VALUES (FROM_UNIXTIME(%s), %s, %s)
                """
            cursor.execute(sql, (dt, row["data_source"], row["missing_project"]))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()
    
def update_new_coins(df, data_source):
    connection = __connect__()
    cursor = get_cursor(connection)
    df = df.astype(object).where(pd.notnull(df), None)
    for i, row in df.iterrows():
        dt = row["mapped_on"].timestamp()
        print(row["mapped_on"], row["missing_project"], data_source)
        try:
            sql = f'update new_symbols_reporting set mapped_on = FROM_UNIXTIME({dt}), cgk_symbol = "{row["symbol"]}", cgk_name="{row["name"]}" where missing_project = "{row["missing_project"]}" and data_source = "{data_source}";'
            cursor.execute(sql)
#             print(sql)
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()
    

In [75]:
insert_new_coins(df)

2021-08-08 00:00:00 tt dYdX
(1062, "Duplicate entry 'dYdX-tt' for key 'new_symbols_reporting.PRIMARY'")
2021-08-08 00:00:00 tt Polymarket
(1062, "Duplicate entry 'Polymarket-tt' for key 'new_symbols_reporting.PRIMARY'")
2021-08-08 00:00:00 tt MetaMask
(1062, "Duplicate entry 'MetaMask-tt' for key 'new_symbols_reporting.PRIMARY'")
2021-08-08 00:00:00 tt Saddle Finance
(1062, "Duplicate entry 'Saddle Finance-tt' for key 'new_symbols_reporting.PRIMARY'")
2021-08-08 00:00:00 tt Autofarm
(1062, "Duplicate entry 'Autofarm-tt' for key 'new_symbols_reporting.PRIMARY'")
2021-08-08 00:00:00 tt Notional Finance
(1062, "Duplicate entry 'Notional Finance-tt' for key 'new_symbols_reporting.PRIMARY'")
2021-08-08 00:00:00 tt Ribbon Finance
(1062, "Duplicate entry 'Ribbon Finance-tt' for key 'new_symbols_reporting.PRIMARY'")
2021-08-08 00:00:00 tt Set Protocol
(1062, "Duplicate entry 'Set Protocol-tt' for key 'new_symbols_reporting.PRIMARY'")
2021-08-08 00:00:00 tt Integral Protocol
(1062, "Duplicate e

## Insert Mappings

In [6]:
mapping_df = pd.read_csv("missing_symbols_mappings/missing_symbols_mappings_2021-08-23.csv")
mapping_df['symbol'] = mapping_df['cgk_symbol']
mapping_df['name'] = mapping_df['cgk_name']
mapping_df = mapping_df[~mapping_df['symbol'].isna()].reset_index(drop=True)
now = datetime.now()
today = datetime(now.year, now.month, now.day)
mapping_df['mapped_on'] = today

In [7]:
mapping_df

,Unnamed: 0,missing_project,data_source,reported_on,mapped_on,cgk_symbol,cgk_name,symbol,name
0,0,AlitaSwap,Debank,2021-08-19,2021-08-23,ALI,Alita,ALI,Alita
1,1,Alpha Tokenomics,Debank,2021-08-20,2021-08-23,ALPHA,Alpha Finance,ALPHA,Alpha Finance
2,2,BurgerSwap,Debank,2021-08-19,2021-08-23,BURGER,Burger Swap,BURGER,Burger Swap
3,3,Dopex,Debank,2021-08-19,2021-08-23,DPX,Dopex,DPX,Dopex
4,5,NAOS,Debank,2021-08-20,2021-08-23,NAOS,NAOS Finance,NAOS,NAOS Finance
5,6,Piggy,Debank,2021-08-21,2021-08-23,PIGGY,Piggy Bank Token,PIGGY,Piggy Bank Token
6,7,Piggy Finance,defistation,2021-08-18,2021-08-23,PIGGY,Piggy Bank Token,PIGGY,Piggy Bank Token
7,8,Saber,solanaprojects,2021-08-19,2021-08-23,SBR,Saber,SBR,Saber
8,9,SCREAM,Debank,2021-08-19,2021-08-23,SCREAM,Scream,SCREAM,Scream


In [8]:
def insert_tt_coin_cgk_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name'], row['missing_project']])
            sql = "INSERT INTO `tt_symbols_cgk` (`symbol`, `name_in_coingecko`, `name_in_tt`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name'], row['missing_project']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

def insert_debank_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name'], row['missing_project']])
            sql = "INSERT INTO `debank_symbols_cgk` (`symbol`, `name_in_coingecko`, `name_in_debank`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name'], row['missing_project']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

def insert_defi_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name'], row['missing_project']])
            sql = "INSERT INTO `defi_symbols_cgk` (`symbol`, `name_in_coingecko`, `name_in_defi`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name'], row['missing_project']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

def insert_defistation_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name'], row['missing_project']])
            sql = "INSERT INTO `defistation_symbols_cgk` (`symbol`, `name_in_coingecko`, `name_in_defistation`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name'], row['missing_project']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

def insert_solana_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name'], row['missing_project']])
            sql = "INSERT INTO `solana_symbols_cgk` (`symbol`, `name_in_coingecko`, `name_in_solana`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name'], row['missing_project']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

In [9]:
tample_mapping = {
    "tt" : insert_tt_coin_cgk_symbol,
    "solanaprojects" : insert_solana_coin_cmc_symbol,
    "defistation" : insert_defistation_coin_cmc_symbol,
    "DefiPlus" : insert_defi_coin_cmc_symbol,
    "Debank" : insert_debank_coin_cmc_symbol,
}

In [10]:
data_sources = mapping_df.data_source.unique().tolist()

In [11]:
for data_source in data_sources:
    print(data_source)
    source_mapping_df = mapping_df[mapping_df['data_source'] == data_source]
    tample_mapping[data_source](source_mapping_df)
    update_new_coins(source_mapping_df, data_source)
    print(source_mapping_df.shape)

Debank
['ALI', 'Alita', 'AlitaSwap']
['ALPHA', 'Alpha Finance', 'Alpha Tokenomics']
['BURGER', 'Burger Swap', 'BurgerSwap']
['DPX', 'Dopex', 'Dopex']
['NAOS', 'NAOS Finance', 'NAOS']
['PIGGY', 'Piggy Bank Token', 'Piggy']
['SCREAM', 'Scream', 'SCREAM']
2021-08-23 00:00:00 AlitaSwap Debank
update new_symbols_reporting set mapped_on = FROM_UNIXTIME(1629676800.0), cgk_symbol = "ALI", cgk_name="Alita" where missing_project = "AlitaSwap" and data_source = "Debank";
2021-08-23 00:00:00 Alpha Tokenomics Debank
update new_symbols_reporting set mapped_on = FROM_UNIXTIME(1629676800.0), cgk_symbol = "ALPHA", cgk_name="Alpha Finance" where missing_project = "Alpha Tokenomics" and data_source = "Debank";
2021-08-23 00:00:00 BurgerSwap Debank
update new_symbols_reporting set mapped_on = FROM_UNIXTIME(1629676800.0), cgk_symbol = "BURGER", cgk_name="Burger Swap" where missing_project = "BurgerSwap" and data_source = "Debank";
2021-08-23 00:00:00 Dopex Debank
update new_symbols_reporting set mapped_on 